In [ ]:
# 29 Mar 2023
# nrobot

In [ ]:
# 0. Dataset description
# -- Graph # of cases vs month over last 10 years

# POSSIBLE Case studies:
# (See if anything interesting comes up...)

# 1. How was use of 'undercover' changed over time
# -- graph by every month for 10 years
# -- in both absolute counts and % counts of total cases
# -- Probably need to avoid coutning multiple times by using the case id?

# 2. How have fines changed over time
# -- (Don't divide by type of offense to start)

# Would be really cool to see impulse change around the robert kraft case (or
# other media news) since that was in florida

# -- If time: Could be more interesting to extract type of offense and then see
# A. how the ratio has changed over time (NOTE to self: like the extreme poverty split by
# region of world graph)
# B. how the amount of the fines depends on the type of offense or other
# covariates 

# So many possibilities!!!! Could also auto assign ethnicities by last name,
# etc. 
# Or pull out stats about the sex workers using date of birth etc.

In [ ]:
# double check which python version running on which computer 
!which python
!uname -n

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pathlib
#import scipy
import calplot

In [ ]:
import altair as alt

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
# - Load raw data: 
cases = pd.read_csv('nogit_data/case_summaries.csv', index_col=0)

# -- put text as column in csv 
files = list( pathlib.Path('nogit_data/text_sample').glob('*.txt'))

corpus = {}
for file in files:
    with open(file, 'r') as f:
        corpus[file.stem] = ''.join(f.readlines())
df_text = pd.DataFrame.from_dict(corpus, orient='index').reset_index()#.columns=['pdf_text']
df_text.columns = ['doc_id', 'pdf_text']

# -- preproces
cases.date = pd.to_datetime(cases['date'])

# -- drop duplicate cases (as PDFs will likely include pages of other pdf)
# keep oldest case by setting date as the index
cases.set_index('date', inplace=True)
cases.drop_duplicates(subset='case', keep='first', inplace=True)
cases.reset_index(inplace=True)

TEST_CASE = 201104833
assert cases[cases.case == TEST_CASE].date.dt.year.values == [2012] # should be 2012, not 2013

annot_text = pd.merge(left=cases, right=df_text, left_on='doc_id', right_on='doc_id')
annot_text.fillna(value={'pdf_text':''}, inplace=True)

# - Create features:
annot_text['undercover'] = annot_text['pdf_text'].apply(lambda x: 'undercover' in x)

# - Plot features:
#print(annot_text.columns)
#print(annot_text.dtypes)

annot_text.set_index('date', inplace=True)
annot_text[(annot_text.index > '1990') & (annot_text.index < '2023')].resample('Y').count()
_data = annot_text[(annot_text.index > '1990') & (annot_text.index < '2023')].resample('Y').count().reset_index()
# -- ?!?! NOTE: altair bar chart is off by one, so 
# -- manually shift the data  when plotting
_data['_year'] = _data.date - pd.DateOffset(years=1) 

In [ ]:
annot_text

In [ ]:
bars = alt.Chart(_data, title='# of Cases Per Year in Florida', ).mark_bar(size=20).encode(
    x=alt.X('_year:T', title='Year'),
    #x=alt.X('x:T', axis=alt.Axis(title='Date', format=("%b"), values=data['date'].dt.year.values)),
    y=alt.Y('case', title='# of Cases'),
    #color='undercover'
)

text = bars.mark_text(
    align='center',
    baseline='middle',
    dy=-10,
    fontSize=15
).encode(
    text='case')


##chart.configure(autosize='fit')
(bars + text + bars.mark_point(size=1000, opacity=0.03, tooltip=alt.TooltipContent("data"))).properties(width=30*30).configure_axis(
    labelFontSize=20,
    titleFontSize=20
).configure_title(
    fontSize=25
)